In [2]:
import tgt
from tgt import read_textgrid
from pandas import DataFrame
import re 
import os
from glob import glob
from pathlib import Path

In [107]:
#大括弧[]で囲むとその中の任意の一文字にマッチする。複数の異なる文字を同じ文字列に置換する場合に使う。
#パターンを|で区切るといずれかのパターンにマッチする

#CEJCで削除するもの "%" "L" "WとWの後半部分"  "Dから)まで"  "<から>まで"　　"Uの部分"  "@の部分"

def clean(data):
    #任意の文字からから任意の文字までを削除 "?"で最短マッチの繰り返しを指定  "."は任意の文字  "*"は０回以上の繰り返し
    #W(言い誤り)を削除
    data = re.sub('W.*?\|',' ',data)
    #D(言いさし)を削除
    data = re.sub('D.*?\)',' ',data)
    #<>（発話行為）を削除
    data = re.sub('\<.*?\>',' ',data)
    data = re.sub('\＜.*?\＞',' ',data)
    #U(聞き取り不明)を削除
    data = re.sub('U.*?\)',' ',data)
    #X(語が不明)を削除
    data = re.sub('X.*?\)',' ',data)
    #K(漢字表記にできない)を削除
    data = re.sub('K.*?\|',' ',data)
    #Y(一般的な読みと発音が異なる)
    data = re.sub('Y.*?\|',' ',data)
    #G(可読性が低い口語表現)
    data = re.sub('\|.*?\)',' ',data)
    #F(フィラー)
    data = re.sub('F.*?\)',' ',data)
    #I(途中に感動紙が挿入)
    data = re.sub('I.*?\)',' ',data)
    #@(転記に対するコメント)
    
    data = data.replace(' ', '')
    data = data.replace(':', 'ー')
    data = data.replace('%', ' ')
    data = data.replace('?', ' ')
    data = data.replace('T', ' ')
    data = data.replace('L', ' ')
    data = data.replace('C', ' ')
    data = data.replace('S', ' ')
    data = data.replace('X', ' ')
    data = data.replace('M', ' ')
    data = data.replace('O', ' ')
    data = data.replace('B', ' ')
    data = data.replace('。', ' ')
    data = data.replace('+',' ')
    data = data.replace('R', ' ')
    data = data.replace('G', ' ')
    data = data.replace('\*', ' ')  
    data = data.replace('(',' ')
    data = data.replace(')',' ')
    return data

In [108]:
#クリーニングを行う


#前ファイルのpathを取得
pathlist = glob('./data/**/**/*transUnit.TextGrid')

#ディレクトリ作成
os.makedirs('./output/pre', exist_ok=True)
os.makedirs('./output/completed', exist_ok=True)

for i in range(len(pathlist)):
    pathname = pathlist[i][-28:]
    tg = read_textgrid(pathlist[i])
    tier_name = tg.get_tier_names()
    tier1 = tg.get_tier_by_name(tier_name[0])
    tier2 = tg.get_tier_by_name(tier_name[1])
    df1 = DataFrame([interval.text for interval in tier1 if interval.text != "pz"], columns=["words"])
    df2 = DataFrame([interval.text for interval in tier2 if interval.text != "pz"], columns=["words"])
    datalist1 = df1["words"]
    #datalist1の@を削除
    for j in range(len(datalist1)):
        if "@"  in datalist1[j]:
            datalist1[j] = re.sub('\@.*',' ',datalist1[j])
        else:
            datalist1[j] = datalist1[j]
    datalist2 = df2["words"]
    #datalist2の@を削除
    for k in range(len(datalist2)):
        if "@"  in datalist2[k]:
            datalist2[k] = re.sub('\@.*',' ',datalist2[k])
        else:
            datalist2[k] = datalist2[k]
    #テキストグリッドファイルをtxtファイルにする(1個目)
    f = open(  './output/pre/'+ pathname + '_1.txt', 'w', encoding='UTF-8')
    f.writelines(datalist1)
    f.close()
    #txtファイルを読み込む
    f = open('./output/pre/'+ pathname + '_1.txt', 'r')
    data1 = f.read()
    f.close()
    data1 = clean(data1)
    f = open(  './output/completed/'+ pathname + '_1.txt', 'w', encoding='UTF-8')
    f.writelines(data1)
    f.close()
    
    #2個目
    f = open(  './output/pre/'+ pathname + '_2.txt', 'w', encoding='UTF-8')
    f.writelines(datalist2)
    f.close()
    #txtファイルを読み込む
    f = open('./output/pre/'+ pathname + '_2.txt', 'r')
    data2 = f.read()
    f.close()
    data2 = clean(data2)
    f = open(  './output/completed/'+ pathname + '_2.txt', 'w', encoding='UTF-8')
    f.writelines(data2)
    f.close()

In [ ]:
#datalist1に@があれば表示
for i in range(len(datalist1)):
    if "@" in datalist1[i]:
        print(datalist1[i])

In [9]:
#@の削除
for i in range(len(datalist1)):
    if "@"  in datalist1[i]:
        datalist1[i] = re.sub('\@.*',' ',datalist1[i])
    else:
        datalist1[i] = datalist1[i]

In [385]:
pathlist2[1][17:25] + pathlist2[1][-6:-4]

'C001_001_2'

In [11]:
pathlist2[23][-6:-4]

'_2'

### 

In [111]:


#pathを取得
pathlist2 = glob('output/completed/*')
pathlist2 = sorted(pathlist2)


#@が入っているpathを表示

for i in range(len(pathlist2)):
    f = open(pathlist2[i], 'r')
    data = f.read()
    if '@' in data:
        print(pathlist2[i][17:25] + pathlist2[i][-6:-4], data.count('@'))
#    else:
#        print("TRUE")

In [425]:
for i in range(len(pathlist2)):
    f = open(pathlist2[i], 'r')
    data = f.read()
    if '＞' in data:
        print(pathlist2[i][17:25] + pathlist2[i][-6:-4])